<a href="https://colab.research.google.com/github/boltonhowes22/image-segmentation-ehg/blob/master/week_01_bolton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Single Neuron Perceptron 
Bolton Howes   
November 2019

The purpose of this assignment is to train a single neuron to predict the cost of grocery store items based on just the total cost and the number of each item purchased (no information abou the cost of each item). 


Begin by importing the data along with a few packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy.matlib

## Just to get rid of 
#import warnings
#warnings.filterwarnings("ignore", category=UserWarning)

# Clone the repository 
!git clone 'https://github.com/boltonhowes22/image-segmentation-ehg.git'

# If you want to view everything in the repository
#!ls image-segmentation-ehg/

#Import csv
df = pd.read_csv('image-segmentation-ehg/week-01//grocery_training_data.csv', header=0)

df.head()

Cloning into 'image-segmentation-ehg'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 131 (delta 56), reused 74 (delta 24), pack-reused 0
Receiving objects: 100% (131/131), 2.02 MiB | 2.25 MiB/s, done.
Resolving deltas: 100% (56/56), done.


,n_bananas,n_apples,n_oranges,n_chocolate,n_kombucha,total_price
0,3,6,3,0,9,49.506000
1,7,7,9,6,7,71.853870
2,9,0,1,9,2,32.135744
3,8,0,1,6,1,21.994054
4,9,1,5,9,4,47.286625


In [0]:
# Some Functions
def extractDigits(lst): 
  return [[el] for el in lst]

def plotCostWeights(cost_list):
    fig = make_subplots(
      rows=1, cols=2,
      subplot_titles=("Cost","Weights"))

    fig.add_trace(go.Scatter(
      x=cost_list['epoch'], y=cost_list['cost'],
      name='Cost',
      mode='markers',
      #marker_color='rgba(152, 0, 0, .8)',
      ),
      row=1, col=1
  )

    fig.add_trace(go.Scatter(
      x=cost_list['epoch'], y=cost_list['weight 1'],
      name='weight 1',
      mode='markers',
      #marker_color='rgba(152, 0, 0, .8)'
      ),
      row=1, col=2
  )

    # Format Axes
    fig.update_xaxes(title_text="Epoch", row=1, col=1)
    fig.update_yaxes(title_text="Cost", row=1, col=1)



    fig.add_trace(go.Scatter(
      x=cost_list['epoch'], y=cost_list['weight 2'],
      name='weight 2',
      mode='markers',
      #marker_color='rgba(152, 0, 0, .8)'
      ),
      row=1, col=2
  )

    fig.add_trace(go.Scatter(
      x=cost_list['epoch'], y=cost_list['weight 3'],
      name='weight 3',
      mode='markers',
      #marker_color='rgba(152, 0, 0, .8)'
      ),
      row=1, col=2
  )

    fig.add_trace(go.Scatter(
      x=cost_list['epoch'], y=cost_list['weight 4'],
      name='weight 4',
      mode='markers',
      #marker_color='rgba(152, 0, 0, .8)'
      ),
      row=1, col=2
  )

    fig.add_trace(go.Scatter(
      x=cost_list['epoch'], y=cost_list['weight 5'],
      name='weight 5',
      mode='markers',
      #marker_color='rgba(152, 0, 0, .8)'
      ),
      row=1, col=2
  )

    # Format Axes
    fig.update_xaxes(title_text="Epoch", row=1, col=1)
    fig.update_yaxes(title_text="Weights", row=1, col=1)
    fig.show
    return fig

def sumSqError(cost_list):

  fw1 = cost_list.iloc[len(cost_list)-1]['weight 1']
  fw2 = cost_list.iloc[len(cost_list)-1]['weight 2']
  fw3 = cost_list.iloc[len(cost_list)-1]['weight 3']
  fw4 = cost_list.iloc[len(cost_list)-1]['weight 4']
  fw5 = cost_list.iloc[len(cost_list)-1]['weight 5']


  sum_sq_error = 0
    
  for i in range(0,len(df)):
    tot = np.sum(df.iloc[i][0:5].values.astype(float) * [fw1, fw2, fw3, fw4, fw5])
    sum_sq_error += (tot-df.iloc[i][5])**2

  return sum_sq_error

def trainTheNeuron(x,y,w,lr,n_epoch, batch_size, end_cost_threshold):
    cost_list = []       # initialize list to store epochs and cost
    y_hat = np.dot(x,w)  # calculate y_hat for first iteration
    num_batches = int(np.floor(len(x)/batch_size))
    for epoch in range(n_epoch):
      for batch in  range(num_batches):
        ## Gradient Descent
        # Calculate Gradient
        dcostdw = 2*0.5*np.dot(-x.T,(y-y_hat))   #  Matrix multiplication dcostdw (mx1) = tranpose(X) (nxm)  * (y - y_hat) (nx1)
        # Calculate New Weights
        w = w - lr*dcostdw                       # w new (3x1) = w (3x1) - dcostdw (3x1)
        # Calculate new estimate from new weights
        y_hat = np.dot(x,w)
        # Calculate new cost                      
        cost = 0.5*np.sum((y - y_hat)**2)        
      cost_list.append([epoch,cost, w[0][0], w[1][0], w[2][0], w[3][0], w[4][0]])
      if epoch%10==0:
          #print("epoch :{:d} cost:{:f}".format(epoch,cost))
          placeholder = 2+2
      if cost <= end_cost_threshold:
          #print("epoch :{:d} cost:{:f}".format(epoch,cost))
          placeholder = 2 + 2
          break
      if True == np.isinf(cost):
          print('ERROR: Gradients exploded, you dummy. Decrease learning rate')
          break
      if True == np.isnan(cost):
          print('ERROR: Gradients exploded, you dummy. Decrease learning rate')
          break
    #print('FINISHED: {} epochs'.format(epoch))
    
    # Put into a dataframe
    cost_list = pd.DataFrame(cost_list,columns=['epoch','cost','weight 1','weight 2','weight 3','weight 4','weight 5'])
    
    cost_list = pd.DataFrame(cost_list,columns=['epoch','cost','weight 1','weight 2','weight 3','weight 4','weight 5'])
    return w,cost_list, epoch

def plotEpochsRequiredBox(x_data,y_data,colors):
  fig = go.Figure()

  for xd, yd, cls in zip(x_data, y_data, colors):
          fig.add_trace(go.Box(
              y=yd,
              name=xd,
              boxpoints='all',
              jitter=0.5,
              whiskerwidth=0.2,
              fillcolor=cls,
              marker_size=4,
              line_width=2.5,
              line_color=cls)
          )

  fig.update_layout(
      title='Number of Training Epochs Required',
      yaxis=dict(
          autorange=True,
          showgrid=True,
          zeroline=True,
          dtick=20,
          gridcolor='rgb(255, 255, 255)',
          gridwidth=1,
          zerolinecolor='rgb(255, 255, 255)',
          zerolinewidth=2,
      ),
      margin=dict(
          l=40,
          r=30,
          b=80,
          t=100,
      ),
      xaxis_title='Batch Size',
      yaxis_title='Epochs',
      paper_bgcolor='rgb(235, 235, 235)',
      plot_bgcolor='rgb(235, 235, 235)',
      showlegend=False
  )

  fig.show()
  return

def reshapeDataForBox(dataframe):
  x_data = ['Batch Size 1', 'Batch Size 10',
            'Batch Size 20', 'Batch Size 30',
            'Batch Size 40', 'Batch Size 50',]


  y0 = (epoch_tally[:,0]).astype(np.int)
  y1 = (epoch_tally[:,1]).astype(np.int)
  y2 = (epoch_tally[:,2]).astype(np.int)
  y3 = (epoch_tally[:,3]).astype(np.int)
  y4 = (epoch_tally[:,4]).astype(np.int)
  y5 = (epoch_tally[:,5]).astype(np.int)

  y_data = [y0, y1, y2, y3, y4, y5]

  colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)',
            'rgba(255, 65, 54, 0.5)', 'rgba(207, 114, 255, 0.5)', 'rgba(127, 96, 0, 0.5)']

  return x_data, y_data, colors




In [22]:
## Input to model 
# Train/Test Split
train,test = train_test_split(df, test_size = 0.2)

# Shape Training Data
train_x = train.iloc[0:len(train)-1, 0:5].values 
train_x = train_x.astype(np.float)

# Shape Testing Data
test_x = test.iloc[0:len(test)-1, 0:5].values
test_x = test_x.astype(np.float)

# Shape Training Target
train_y = train.iloc[0:len(train)-1,5]
train_y = list(train_y.astype(np.float))
train_y = extractDigits(train_y)


# Shape Testing Target
test_y = test.iloc[0:len(test)-1,5]
test_y = list(test_y.astype(np.float))
test_y = extractDigits(test_y)

# Hyperparameters
w = np.random.rand(5,1)
n_epoch = 100
lr = 0.0001
batch_size = 20
end_cost_threshold = 0.001

w,cost_list, epoch = trainTheNeuron(train_x,train_y,w,lr,n_epoch, batch_size, end_cost_threshold)
#print('Sum of Square Errors: {}'.format(sumSqError(cost_list)/len(cost_list)))

plotCostWeights(cost_list)


In [0]:
## Now I want to see the effect of batch size on how quickly the neuron learns the weights. 
## I will test different batch sizes and see how many epochs it takes to reach the a cost of 0.001
batch_size_list = [1, 10, 20, 30, 40, 50]
num_weights = 5
num_iterations = 1000
weight_array = 5* np.random.rand(num_weights, num_iterations)
n_epoch = 200
lr = 0.0001
end_cost_threshold = 0.001

# Preallocate a couple of variables
epoch_tally = np.zeros((num_iterations,len(batch_size_list)))
batch_num = 0

# Begin the training loop
for batch_size in batch_size_list:
  run_num = 0
  batch_num += 1
  for weight_n in range(num_iterations):
    run_num += 1
    input_weight = weight_array[:,weight_n].reshape(5,1)
    w,cost_list, epoch_tally[run_num-1, batch_num-1] = trainTheNeuron(train_x,train_y,input_weight,lr,n_epoch, batch_size, end_cost_threshold = 0.001)


## collect number of epochs required to reach cost of 0.001 and put in an dataframe
epoch_df = pd.DataFrame({'Batch Size 1': epoch_tally[:, 0], 'Batch Size 10': epoch_tally[:, 1], 'Batch Size 20': epoch_tally[:, 2], 'Batch Size 30': epoch_tally[:, 3], 'Batch Size 40': epoch_tally[:, 4], 'Batch Size 50': epoch_tally[:, 5]})

In [24]:
x_data,y_data,colors = reshapeDataForBox(epoch_df)

plotEpochsRequiredBox(x_data,y_data,colors)



In [1]:
## What if we repeat this experiment, but add noise to the target??
# Shape Training Target
train_y = train.iloc[0:len(train)-1,5]
train_y = list(train_y.astype(np.float))
train_y = extractDigits(train_y)

## Add some noise
# Preallocate
train_noiseY = np.zeros((len(train_y),1))
for i in range(len(train_y)):
  train_noiseY[i] = np.random.normal(train_y[i],.08,1)

# Hyperparameters
batch_size_list = [1, 10, 20, 30, 40, 50]
num_weights = 5
num_iterations = 100
weight_array = 5* np.random.rand(num_weights, num_iterations)
n_epoch = 500
lr = 0.0001
end_cost_threshold = .001

# Preallocate a couple of variables
epoch_tally = np.zeros((num_iterations,len(batch_size_list)))
batch_num = 0

# Begin the training loop
for batch_size in batch_size_list:
  run_num = 0
  batch_num += 1
  for weight_n in range(num_iterations):
    run_num += 1
    input_weight = weight_array[:,weight_n].reshape(5,1)
    w,cost_list, epoch_tally[run_num-1, batch_num-1] = nn_batch_generalized(train_x,train_noiseY,input_weight,lr,n_epoch, batch_size, end_cost_threshold)


## collect number of epochs required to reach cost of 0.001 and put in an dataframe
epoch_noisey_df = pd.DataFrame({'Batch Size 1': epoch_tally[:, 0], 'Batch Size 10': epoch_tally[:, 1], 'Batch Size 20': epoch_tally[:, 2], 'Batch Size 30': epoch_tally[:, 3], 'Batch Size 40': epoch_tally[:, 4], 'Batch Size 50': epoch_tally[:, 5]})

NameError: ignored

In [163]:
x_data,y_data,colors = reshapeDataForBox(epoch_noisey_df)

plotEpochsRequiredBox(x_data,y_data,colors)

In [157]:
cost_list

,epoch,cost,weight 1,weight 2,weight 3,weight 4,weight 5
0,0,3152.628551,1.562656,1.416233,0.282794,2.558170,4.443622
1,1,2606.653393,1.480120,1.545927,0.408597,2.472042,4.341030
2,2,2161.624239,1.404715,1.662406,0.521929,2.398337,4.244951
3,3,1797.942606,1.336013,1.767380,0.624348,2.335751,4.155264
4,4,1499.946495,1.273341,1.862056,0.716955,2.282873,4.071548
...,...,...,...,...,...,...,...
495,495,0.268294,0.492133,2.858195,1.628663,2.251516,2.890134
496,496,0.268294,0.492133,2.858195,1.628663,2.251516,2.890134
497,497,0.268294,0.492133,2.858195,1.628663,2.251516,2.890134
498,498,0.268294,0.492133,2.858195,1.628663,2.251516,2.890134
